In [1]:
import pandas as pd
import numpy as np

# Настройки генерации
NUM_SAMPLES = 100000  # Количество записей
TARGET_DIAMETER = 1.75
TOLERANCE = 0.05  # Допуск +/- 0.05 мм
MAX_OVALITY = 0.05 # Максимальная овальность

def generate_filament_data(n_samples):
    # --- 1. Генерация базовых параметров процесса (ВХОДЫ) ---
    
    # Температура экструдера (Норма ~210, но иногда скачет)
    # Генерируем нормальное распределение и добавляем случайные сбои
    extruder_temp = np.random.normal(loc=210, scale=21, size=n_samples)
    
    # Скорость шнека (RPM)
    screw_speed = np.random.normal(loc=30, scale=3, size=n_samples)
    
    # Давление расплава (Зависит от температуры и скорости + шум)
    # Физика: ниже температура -> выше вязкость -> выше давление
    melt_pressure = (210 / extruder_temp) * 100 + (screw_speed * 0.5) + np.random.normal(0, 2, n_samples)
    
    # Влажность сырья (Обычно сухая < 0.1%, но иногда попадается влажная партия)
    # Используем экспоненциальное распределение для имитации редких выбросов влажности
    moisture = np.random.exponential(scale=0.05, size=n_samples) 
    
    # Скорость тянущего устройства (Puller)
    # В идеале она синхронизирована со шнеком, но добавим рассинхрон
    puller_speed = screw_speed * 0.5 + np.random.normal(0, 0.5, n_samples)
    
    # Температура охлаждающей воды
    water_temp = np.random.normal(loc=40, scale=4, size=n_samples)
    
    # Влажность воздуха в цеху (влияет слабо, но параметр есть)
    air_humidity = np.random.normal(loc=45, scale=5, size=n_samples)

    # --- 2. Симуляция сбоев (искусственно портим часть данных) ---
    # Выбираем случайные индексы для имитации аварий
    
    # Сценарий А: Сбой нагревателя (температура падает)
    idx_cold = np.random.choice(n_samples, size=int(n_samples * 0.05), replace=False)
    extruder_temp[idx_cold] -= np.random.uniform(10, 30, size=len(idx_cold))
    
    # Сценарий Б: Рассинхрон (тянет слишком быстро)
    idx_pull = np.random.choice(n_samples, size=int(n_samples * 0.05), replace=False)
    puller_speed[idx_pull] += np.random.uniform(1.0, 3.0, size=len(idx_pull))

    # Сценарий В: Очень влажное сырье
    idx_wet = np.random.choice(n_samples, size=int(n_samples * 0.05), replace=False)
    moisture[idx_wet] += np.random.uniform(0.3, 0.8, size=len(idx_wet))

    # --- 3. Расчет выходных метрик качества (ВЫХОДЫ) ---
    
    # Расчет Диаметра (упрощенная формула физики)
    # Диаметр зависит от соотношения Скорости Шнека к Скорости Тяги и Температуры
    ratio = (screw_speed * 0.5) / puller_speed # В идеале 1.0
    thermal_expansion = (extruder_temp - 210) * 0.001 # Влияние тепла
    
    # Итоговый диаметр с добавлением шума
    diameter = TARGET_DIAMETER * ratio + thermal_expansion + np.random.normal(0, 0.01, n_samples)
    
    # Расчет Овальности
    # Зависит от влажности (пузыри) и температуры воды (деформация)
    ovality_base = np.abs(np.random.normal(0, 0.01, n_samples))
    ovality_moisture = moisture * 0.1 # Влага сильно влияет на форму
    ovality_water = np.abs(water_temp - 40) * 0.002 # Плохое охлаждение портит форму
    
    ovality = ovality_base + ovality_moisture + ovality_water

    # --- 4. Маркировка БРАК / НЕ БРАК (Target Label) ---
    
    # Условия брака:
    # 1. Диаметр вышел за допуск (1.70 - 1.80)
    # 2. Овальность больше 0.05
    
    is_defect = np.zeros(n_samples, dtype=int)
    
    mask_diameter_fail = (diameter > (TARGET_DIAMETER + TOLERANCE)) | (diameter < (TARGET_DIAMETER - TOLERANCE))
    mask_ovality_fail = ovality > MAX_OVALITY
    
    is_defect[mask_diameter_fail | mask_ovality_fail] = 1

    # Собираем в DataFrame
    df = pd.DataFrame({
        'Extruder_Temp_C': extruder_temp,     # Температура экструдера
        'Screw_Speed_RPM': screw_speed,       # Скорость шнека
        'Melt_Pressure_Bar': melt_pressure,   # Давление
        'Puller_Speed_M_min': puller_speed,   # Скорость протяжки
        'Water_Temp_C': water_temp,           # Температура воды
        'Material_Moisture_Pct': moisture,    # Влажность сырья
        'Air_Humidity_Pct': air_humidity,     # Влажность воздуха
        'Output_Diameter_mm': diameter,       # Диаметр (Выход)
        'Output_Ovality_mm': ovality,         # Овальность (Выход)
        'Defect_Flag': is_defect              # Целевая переменная (0 или 1)
    })
    
    return df

# Генерация
df_filament = generate_filament_data(NUM_SAMPLES)

# Округление для красивого вида
df_filament = df_filament.round(3)

# Проверка баланса классов
print("Статистика по датасету:")
print(df_filament['Defect_Flag'].value_counts(normalize=True))
print(f"\nВсего записей: {len(df_filament)}")

# Сохранение в CSV
df_filament.to_csv('filament_production_data_noisy.csv', index=False)
print("Файл 'filament_production_data.csv' успешно создан.")

Статистика по датасету:
Defect_Flag
0    0.51641
1    0.48359
Name: proportion, dtype: float64

Всего записей: 100000
Файл 'filament_production_data.csv' успешно создан.
